# Ch1 : python data model

数据模型其实是对Python框架的描述，它规范了这门语言自身构建模块的接口，这些模块包括但不限于序列、迭代器、函数、类和上下文管理器。

magic methods是python中的特殊方法，它们的名字以双下划线开头和结尾，比如`__getitem__`。这些方法是python解释器调用的，而不是我们自己调用的。

- 迭代
- 集合类
- 属性访问
- 运算符重载
- 函数和方法的调用
- 对象的创建和销毁
- 字符串表示形式和格式化
- 管理上下文 (aka `with` block)

## 1.1 一摞Python风格的纸牌

How to use `__getitem__` and `__len__` methods

In [41]:
# frenchdeck.py
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]
    
    # 这一方法破坏了类的封装性使得我们可以直接对_cards进行操作，这个类变为mutable对象
    def __setitem__(self, position, value):
        self._cards[position] = value

`collections.namedtuple` is a factory function for creating simple tuple subclasses with named fields (no methods required). Like database entries.

In [42]:
beer_card = Card('7','diamonds')
beer_card

Card(rank='7', suit='diamonds')

In [43]:
deck = FrenchDeck()
len(deck)

52

In [44]:
print(deck[0])
print(deck[-1])

Card(rank='2', suit='spades')
Card(rank='A', suit='hearts')


`len()` and `[]` will invoke `__len__` and `__getitem__` methods respectively.

What's more, there is no need to implement a new method for random choice, just use `random.choice()`. Because it will call `__getitem__` method.

In [45]:
from random import choice
print(choice(deck))
print(choice(deck))

Card(rank='3', suit='hearts')
Card(rank='7', suit='spades')


since `__getitem__` gives `[]` operation to `self._cards`, the deck class support slicing operation. Also, it supports iteration.

In [46]:
for card in reversed(deck):
    print(card)

Card(rank='A', suit='hearts')
Card(rank='K', suit='hearts')
Card(rank='Q', suit='hearts')
Card(rank='J', suit='hearts')
Card(rank='10', suit='hearts')
Card(rank='9', suit='hearts')
Card(rank='8', suit='hearts')
Card(rank='7', suit='hearts')
Card(rank='6', suit='hearts')
Card(rank='5', suit='hearts')
Card(rank='4', suit='hearts')
Card(rank='3', suit='hearts')
Card(rank='2', suit='hearts')
Card(rank='A', suit='clubs')
Card(rank='K', suit='clubs')
Card(rank='Q', suit='clubs')
Card(rank='J', suit='clubs')
Card(rank='10', suit='clubs')
Card(rank='9', suit='clubs')
Card(rank='8', suit='clubs')
Card(rank='7', suit='clubs')
Card(rank='6', suit='clubs')
Card(rank='5', suit='clubs')
Card(rank='4', suit='clubs')
Card(rank='3', suit='clubs')
Card(rank='2', suit='clubs')
Card(rank='A', suit='diamonds')
Card(rank='K', suit='diamonds')
Card(rank='Q', suit='diamonds')
Card(rank='J', suit='diamonds')
Card(rank='10', suit='diamonds')
Card(rank='9', suit='diamonds')
Card(rank='8', suit='diamonds')
Card(r

Iteration is usually implicit. If a collection has no `__contains__` method, the `in` operator does a sequential scan. 

In [47]:
Card('Q', 'hearts') in deck 

True

In [48]:
Card('7', 'beasts') in deck 

False

Next, we will implement sorting

In [49]:
suit_values = dict(spades = 3, hearts = 2, diamonds = 1, clubs = 0)

def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]

In [50]:
from random import shuffle
shuffle(deck) # also used __getitem__

for card in deck:
    print(card)

for card in sorted(deck, key = spades_high):
    print(card)

Card(rank='4', suit='clubs')
Card(rank='9', suit='spades')
Card(rank='A', suit='spades')
Card(rank='9', suit='hearts')
Card(rank='2', suit='spades')
Card(rank='10', suit='diamonds')
Card(rank='4', suit='spades')
Card(rank='10', suit='clubs')
Card(rank='3', suit='clubs')
Card(rank='2', suit='diamonds')
Card(rank='K', suit='spades')
Card(rank='Q', suit='diamonds')
Card(rank='A', suit='hearts')
Card(rank='A', suit='diamonds')
Card(rank='5', suit='spades')
Card(rank='10', suit='hearts')
Card(rank='8', suit='hearts')
Card(rank='8', suit='spades')
Card(rank='6', suit='spades')
Card(rank='3', suit='diamonds')
Card(rank='2', suit='clubs')
Card(rank='3', suit='spades')
Card(rank='6', suit='clubs')
Card(rank='5', suit='diamonds')
Card(rank='J', suit='clubs')
Card(rank='4', suit='diamonds')
Card(rank='7', suit='spades')
Card(rank='9', suit='clubs')
Card(rank='J', suit='spades')
Card(rank='10', suit='spades')
Card(rank='Q', suit='hearts')
Card(rank='4', suit='hearts')
Card(rank='K', suit='hearts')

## 1.2 如何使用特殊方法
特殊方法的存在是为了被Python解释器调用的，你自己并不需要调用它们。也就是说没有`my_object.__len__()` 这种写法，而应该使用`len(my_object)`。

通过内置的函数（例如len、iter、str，等等）来使用特殊方法是最好的选择。这些内置函数不仅会调用特殊方法，通常还提供额外的好处，而且对于内置的类来说，它们的速度更快。

In [51]:
# vector2d.py
from math import hypot

class Vector:

    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def __repr__(self):
        return 'Vector(%r, %r)' % (self.x, self.y)

    def __abs__(self):
        return hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y)

    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)


### 1.2.2 `__repr__` 字符串表示形式

很像java的`toString()`方法，用于返回一个对象的字符串表示形式。

`__repr__` 和 `__str__` 的区别在于，后者是在str()函数被使用，或是在用print函数打印一个对象的时候才被调用的，并且它返回的字符串对终端用户更友好。

### 1.2.4 自定义的布尔值
尽管Python 里有bool类型，但实际上任何对象都可以用于需要布尔值的上下文中（比如if 或while 语句，或者and、or和not运算符）。默认情况下，我们自己定义的类返回的实例总被认为是True

`vector.__bool__`也可以改写为以下形式：
```python
def __bool__(self):
    return bool(self.x or self.y)
```

# 1.4 为什么len不是普通方法
`len`是一个内置函数，而不是一个普通方法，这是为了让Python自带的数据结构可以走后门，直接从底层的C语言中获得长度，速度非常快。


*延申阅读*

元对象